In [ ]:
!pip install osmnx

Usar "!pip install osmnx" para baixar a biblioteca *osmnx*, que, ao contrário das demais bibliotecas necessárias, não está previamente instalada no Colab. Se, posteriormente, ao tentar importar a biblioteca *osmnx*, aparecer o erro:

"**AttributeError: module 'numpy' has no attribute '_no_nep50_warning'**"

Isso pode acontecer caso a versão da biblioteca *numpy* seja 1.24.0.

Usar os comandos:


*   Digitar "y" quando solicitado.



In [ ]:
!pip uninstall numpy 
!pip install numpy<1.23.0

Se, ao importar a biblioteca *osmnx*, aparecer o erro:

"**ImportError: cannot import name 'BoundsOp' from 'shapely.coords'**

Usar os comandos:

*   Digitar "y" quando solicitado.




In [ ]:
!pip uninstall shapely
!pip install shapely

Se, durante o código, no trecho referente à produção das imagens, aparecer o erro:

"**Cannot import name '_png' from matplotlib**"

Isso pode acontecer caso a versão da biblioteca *matplotlib* seja 3.7.0.

Usar os comandos:
*   Digitar "y" quando solicitado.




In [ ]:
!pip uninstall matplotlib
!pip install matplotlib==3.1.3

Primeiramente, importar a classe drive do módulo *google.colab* para ter acesso aos arquivos presentes no seu Google Drive. Clicar em “Conectar ao Google Drive”, escolher a conta com a qual vai fazer login e permitir acesso ao Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import osmnx as ox
import networkx as nx
import pandas as pd
import matplotlib
import openpyxl
import numpy as np

O código Python abaixo tem como objetivo calcular estatísticas de acessibilidade da malha urbana para cidades brasileiras, utilizando a **biblioteca OSMnx**.

Primeiramente, são importadas as bibliotecas necessárias para a execução do código, tais como pandas, openpyxl, osmnx e matplotlib.

Em seguida, **é criado um arquivo Excel (chamado de "cidades_comDM_stats.xlsx")** utilizando a biblioteca pandas e o engine openpyxl.

O arquivo **"BR_cidades_comDM_20230301_UTF8.csv"** é lido e armazenado na variável "cidades".

Essa tabela contém informações de cidades brasileiras como nome, UF, latitude, longitude, entre outras.

A seguir, é **definida uma função chamada "calculo_malha"** que recebe três parâmetros:

1. "cidades" (a tabela de cidades),

2. "malha" (o tipo de malha a ser calculada, podendo ser "all" ou "bike") e

3. "planilha" (o nome da planilha do arquivo Excel onde serão armazenados os resultados).

Essa função é responsável por calcular as estatísticas de acessibilidade para cada cidade, armazenar os resultados em um dicionário e, por fim, escrever esses resultados em uma planilha Excel.

Dentro da função "calculo_malha", é criado um dicionário vazio chamado "df_final" que será preenchido com os resultados de cada cidade.

Além disso, são criados mais dois dicionários vazios chamados "df_counts" e "df_proportions" que serão utilizados para armazenar algumas estatísticas específicas.

A partir de um loop que itera sobre cada cidade na tabela "cidades", a função calcula as estatísticas para cada cidade.

Para cada cidade, também **é gerada uma imagem da malha urbana** utilizando a função "ox.plot_figure_ground", que salva a imagem em um **formato .png na pasta "images".**

Em seguida, é calculada a área da cidade e as estatísticas básicas utilizando a função "ox.basic_stats". Os resultados são armazenados no dicionário "df_final".

Depois de calcular as estatísticas para todas as cidades, o dicionário "df_final" é dividido em dois dicionários separados: "df_counts" e "df_proportions".
O primeiro armazena as contagens de trechos por nó da malha urbana, enquanto o segundo armazena as proporções de trechos por nó.
Esses dicionários são criados a partir de um loop que itera sobre cada cidade na tabela "cidades" e preenche os dicionários com as estatísticas correspondentes.

Em seguida, são realizadas algumas transformações nos nomes das colunas dos dicionários para facilitar a análise posterior.
Após essas transformações, as colunas dos dicionários "df_counts" e "df_proportions" são combinadas e adicionadas ao dicionário "df_final".

Por fim, os resultados são escritos em uma planilha Excel utilizando a função "to_excel" da biblioteca pandas.

O nome da planilha é determinado pelo parâmetro "planilha" passado para a função "calculo_malha".

O arquivo *.xlsx* só terá algum dado dentro dele ao final de toda iteração de cada malha. De outra forma, ficará vazio.

In [ ]:
#Criação do arquivo final que irá armezenar todos os dados
arquivo = pd.ExcelWriter('cidades_comDM_stats.xlsx', engine='openpyxl')

#Leitura dos dados do csv presente no drive
cidades = pd.read_csv("/content/drive/MyDrive/CNPQ - Mobilidade Ativa e Saúde/3_Trabalho/WP1_MACRO_BE/BE/BR_cidades_comDM_20230301_UTF8.csv",
                      sep = ';', decimal=',')

#Função que produz as imagens e calcula as estatísticas
def calculo_malha(cidades, malha, planilha):
    #Define os dicionários que serão usados. O primeiro contém a chave que armazenará as cidades
    df_final = {'Cidade': [], 'Código IBGE': []}
    df_counts = {}
    df_proportions = {}

    ## CÓDIGO PARA SALVAR OS PNG EM ALGUMA PASTA PERENE

    #Define o nome da pasta em que a foto será salva, o formato das imagens e os pontos por polegada
    img_pasta = "images"
    formato = "png"
    ppp = 96

    #Loop que corre as cidades no csv do drive
    for nome in cidades['municipio_formatado']:
        
        #Define a string para geração dos grafos pegando os valores (Estado e País)
        # correspondentes a mesma linha da cidade
        lugar = ('{}, {}, {}'.format
                 ((cidades.loc[cidades['municipio_formatado'] == nome]['municipio_formatado'].values),
                  (cidades.loc[cidades['municipio_formatado'] == nome]['uf_completo'].values),
                  (cidades.loc[cidades['municipio_formatado'] == nome]['pais'].values)))
        
        #Adiciona o nome da cidade e código IBGE à chave 'Cidade' e 'Código IBGE"
        df_final['Cidade'].append(nome)
        df_final['Código IBGE'].append(cidades.loc[cidades['municipio_formatado'] == nome]['ibge'].values)
                
        #Geração do grafo
        graph = ox.graph_from_place(lugar, network_type = malha)
        
        gdf = ox.geocode_to_gdf(lugar)
        area = ox.project_gdf(gdf).unary_union.area
        
        #Cálculo das estatísticas básicas a partir da área
        stats = ox.basic_stats(graph, area = area)
        
        #Acrescentar as estatísticas ao dicionário
        print('{}: '.format(lugar))
        for k in stats.items():
            df_final.setdefault(k[0], []).append(k[1])
            
            print('{} = {}'.format(k[0], k[1]))

        #Input para imprimir ou não os cálculos de distância dos nodos
        distancia_nodos = str(input("Imprimir closenness centrality e betweenness centrality para {}? (s/n) ".format(lugar))).lower().strip()
        if distancia_nodos == 's':
          print('Closeness centrality: {}'.format(np.mean(list(nx.closeness_centrality(graph).values()))))
          print('Betweenness centrality: {}'.format(np.mean(list(nx.betweenness_centrality(graph, weight = 'length').values()))))      

        #Caso a malha não seja 'bike', input para plotar ou não figura da malha 
        if malha != 'bike':

          plot_imagem = str(input("Imprimir imagem da malha de {}? (s/n) ".format(lugar))).lower().strip()
          if plot_imagem == 's':
              fp = "./{}/{}_{}.{}".format(img_pasta, lugar, malha, formato)
              fig, ax = ox.plot_figure_ground(
                address = lugar,
                dist = 1600,
                network_type= malha,
                default_width = 1, 
                filepath = fp,
                dpi = ppp,
                save = True,
                close=True)

    #Loops para separar os dicionários das variáveis 
    # 'streets_per_node_counts' e 'streets_per_node_proportions'
    for j in df_final['streets_per_node_counts']:
           for counts in j.items():
               df_counts.setdefault((counts[0]), [])    
                
    for key1 in df_counts.keys():
            df_counts[key1] = [
                d1.get(key1, 0) for d1 in df_final['streets_per_node_counts']
                ]           
                
    for l in df_final['streets_per_node_proportions']:
            for proportions in l.items():
                df_proportions.setdefault((proportions[0]), [])
                
    for key2 in df_proportions.keys():
            df_proportions[key2] = [   
                d2.get(key2, 0) for d2 in df_final['streets_per_node_proportions']
                ]           
      
    for nome_counts in list(df_counts.keys()).copy():
        df_counts["{} way_int_count"
                  .format(nome_counts)] = df_counts[nome_counts]
        del df_counts[nome_counts]
      
    for nome_proportions in list(df_proportions.keys()).copy():
        df_proportions["{} way_int_prop"
                  .format(nome_proportions)] = df_proportions[nome_proportions]
        del df_proportions[nome_proportions]

    #Deletar as colunas originais
    del df_final['streets_per_node_counts']
    del df_final['streets_per_node_proportions']

    #Adicionar os dicionários uns aos outros
    df_counts.update(df_proportions)
    df_final.update(df_counts)

    #Transformar em dataframe
    indicadores = pd.DataFrame.from_dict(df_final)

    #Transformar em arquivo excel
    indicadores.to_excel(arquivo, sheet_name = planilha)


#Malha geral
calculo_malha(cidades, 'all', 'planilha1')

#Malha cicloviária
bike_malha = str(input('Calcular para a malha cicloviária? (s/n) ')).lower().split()
if bike_malha == 's':
  calculo_malha(cidades, 'bike', 'planilha2')

#Opção para deletar os dicionários
#del df_counts, df_proportions, df_final

arquivo.close()